# 一、pandas基本数据结构
#### 存储一维 values 的 Series 
#### 存储二维 values 的 DataFrame

In [2]:
import pandas as pd

## 1. Series

### Series 一般由四个部分组成，分别是序列的值 data 、索引 index 、存储类型 dtype 、序列的名字 name 。其中，索引也可以指定它的名字，默认为空。

In [4]:
s = pd.Series(data = [100, 'a', {'dic1':5}],# 序列的值
              index = pd.Index(['id1', 20, 'third'], name='my_idx'),# 索引 index
              dtype = 'object',# 存储类型 dtyp
              name = 'my_name'#序列的名字 name
             )
s

my_idx
id1              100
20                 a
third    {'dic1': 5}
Name: my_name, dtype: object

## object类型
### object 代表了一种混合类型，正如上面的例子中存储了整数、字符串以及 Python 的字典数据结构。此外，目前 pandas 把纯字符串序列也默认认为是一种 object 类型的序列，但它也可以用 string 类型存储

## 2. dataframe

### DataFrame 在 Series 的基础上增加了列索引，一个数据框可以由二维的 data 与行列索引来构造

In [5]:
data = [[1, 'a', 1.2], [2, 'b', 2.2], [3, 'c', 3.2]]
df = pd.DataFrame(data = data,
                  index = ['row_%d'%i for i in range(3)],
                  columns=['col_0', 'col_1', 'col_2'])
df

,col_0,col_1,col_2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


# 二、常用基本函数

In [9]:
df = pd.read_csv('../data/learn_pandas.csv')
df_demo = df[['Height', 'Weight']]

### 1. 聚合函数

### quantile, count, idxmax 这三个函数，它们分别返回的是分位数、非缺失值个数、最大值对应的索引

In [10]:
df_demo.quantile(0.75)

Height    167.5
Weight     65.0
Name: 0.75, dtype: float64

In [11]:
df_demo.count()

Height    183
Weight    189
dtype: int64

In [12]:
df_demo.idxmax() # idxmin是对应的函数

Height    193
Weight      2
dtype: int64

#### 上面这些所有的函数，由于操作后返回的是标量，所以又称为聚合函数，它们有一个公共参数 axis ，默认为0代表逐列聚合，如果设置为1则表示逐行聚合

### 2. 唯一值函数

#### unique 唯一值组成的列表
#### nunique 唯一值的个数
#### drop_duplicates 多个列组合的唯一值

### 3. 排序函数

#### 值排序 sort_values df_demo.sort_values(['Weight','Height'],ascending=[True,False])
#### 索引排序 sort_index df_demo.sort_index(level=['Grade','Name'],ascending=[True,False])

### 4. 窗口对象
#### 1. 滑窗对象
#### 要使用滑窗函数，就必须先要对一个序列使用 .rolling 得到滑窗对象，其最重要的参数为窗口大小 window 。

#### shift, diff, pct_change 是一组类滑窗函数，它们的公共参数为 periods=n ，默认为1，分别表示取向前第 n 个元素的值、与向前第 n 个元素做差（与 Numpy 中不同，后者表示 n 阶差分）、与向前第 n 个元素相比计算增长率。这里的 n 可以为负，表示反方向的类似操作。

#### 2. 扩张窗口
#### 扩张窗口又称累计窗口，可以理解为一个动态长度的窗口，其窗口的大小就是从序列开始处到具体操作的对应位置，其使用的聚合函数会作用于这些逐步扩张的窗口上

In [14]:
s = pd.Series([1, 3, 6, 10])

print(s.expanding().mean())
print(s.cummax())
print(s.cumsum())
print(s.cumprod())

0    1.000000
1    2.000000
2    3.333333
3    5.000000
dtype: float64
0     1
1     3
2     6
3    10
dtype: int64
0     1
1     4
2    10
3    20
dtype: int64
0      1
1      3
2     18
3    180
dtype: int64


# 三、 练习题

### Ex1：口袋妖怪数据集

In [16]:
df = pd.read_csv('../data/pokemon.csv')

df.head(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80


#### 1. 对 HP, Attack, Defense, Sp. Atk, Sp. Def, Speed 进行加总，验证是否为 Total 值。

In [29]:
(df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].sum(axis = 1) != df['Total']).sum()

0

#### 2. 对于 # 重复的妖怪只保留第一条记录，解决以下问题：

In [30]:
dp_drop_dup = df.drop_duplicates('#', keep='first')

#### a. 求第一属性的种类数量和前三多数量对应的种类

In [32]:
dp_drop_dup['Type 1'].nunique()

18

In [33]:
dp_drop_dup['Type 1'].value_counts().index[:3]

Index(['Water', 'Normal', 'Grass'], dtype='object')

#### b. 求第一属性和第二属性的组合种类

In [37]:
dp_drop_dup.drop_duplicates(['Type 1','Type 2']).shape[0]

143

#### c. 求尚未出现过的属性组合

In [86]:
from itertools import product
type1 = df['Type 1'].unique()
type2 = df['Type 2'].unique()
all_ = set(product(*[type1,type2]))
now_ = set(df[['Type 1','Type 2']].apply(lambda x: tuple(x), axis=1).values.tolist())

In [87]:
len(all_.difference(now_))

188

#### 3. 按照下述要求，构造 Series 

#### a. 取出物攻，超过120的替换为 high ，不足50的替换为 low ，否则设为 mid

In [98]:
df['Attack'].mask(df['Attack']>120,'high').mask(df['Attack']<50,'low').mask((df['Attack']<120) & (df['Attack']>50),'mid')

0       low
1       mid
2       mid
3       mid
4       mid
       ... 
795     mid
796    high
797     mid
798    high
799     mid
Name: Attack, Length: 800, dtype: object

#### b. 取出第一属性，分别用 replace 和 apply 替换所有字母为大写

In [101]:
df['Type 1'].replace({i:str.upper(i) for i in df['Type 1'].unique()})

0        GRASS
1        GRASS
2        GRASS
3        GRASS
4         FIRE
        ...   
795       ROCK
796       ROCK
797    PSYCHIC
798    PSYCHIC
799       FIRE
Name: Type 1, Length: 800, dtype: object

In [105]:
df['Type 1'].apply(lambda x:str.upper(x))

0        GRASS
1        GRASS
2        GRASS
3        GRASS
4         FIRE
        ...   
795       ROCK
796       ROCK
797    PSYCHIC
798    PSYCHIC
799       FIRE
Name: Type 1, Length: 800, dtype: object

#### c. 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到 df 并从大到小排序

In [107]:
import numpy as np
df['Deviation'] = df[['HP', 'Attack', 'Defense', 'Sp. Atk',
                     'Sp. Def', 'Speed']].apply(lambda x:np.max(
                     (x-x.median()).abs()), 1)
df.sort_values('Deviation', ascending=False).head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Deviation
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,215.0
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,207.5
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,190.0
333,306,AggronMega Aggron,Steel,NaN,630,70,140,230,60,80,50,155.0
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,145.0


### Ex2：指数加权窗口

In [112]:
np.random.seed(0)
s = pd.Series(np.random.randint(-1,2,30).cumsum())
s.ewm(alpha = 0.2).mean().head()

0   -1.000000
1   -1.000000
2   -1.369004
3   -1.552486
4   -1.661767
dtype: float64

In [109]:
def ewm_func(x, alpha=0.2):
    win = (1-alpha)**np.arange(x.shape[0])[::-1]
    res = (win*x).sum()/win.sum()
    return res
s.expanding().apply(ewm_func).head()

0   -1.000000
1   -1.000000
2   -1.409836
3   -1.609756
4   -1.725845
dtype: float64